# 3: Build a neural network with AutoKeras

In [2]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import plotly.express as px
import scipy.sparse
import os

# import keras
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential, Model 
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, GlobalMaxPool1D, Flatten, Input, concatenate, Dropout, Activation, regularizers, BatchNormalization


from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix

print('Autokeras needs TF>=2')
print('TF version:',tf.__version__)


TF version: 1.15.0


In [36]:
cwd = os.getcwd()
print(cwd)
os.chdir(cwd)

C:\Users\brizio\Documents\PythonNB\FICOchallenge


## Load prepped data
- Normal Scaling of numeric variables (prep_option = 1)
- Binning (following Rudin) and one hot encoding (prep_option = 2)
- Binning and applying WOE, calculating WOE on Rudin's bins (prep_option = 3)
- Binning and applying WOE, following Rudin (prep_option = 4)

In [1]:
prep_option = 4

In [62]:
if prep_option == 1:
    data_path = "Data/Scaled_data.csv"
if prep_option == 2:
    data_path = "Data/Bin_Encoded_data_v2.csv"
if prep_option == 3:
    data_path = "Data/WOE_data.csv"
if prep_option == 4:
    data_path = "Data/WOE_Rud_data.csv"
    
ori_path = "Data/heloc_dataset_v1.csv"

CLASS = 'RiskPerformance' 

data = pd.read_csv(ori_path)
X1 = pd.read_csv(data_path)
y = pd.read_csv("Data/y_data.csv")

print('Target: Bad (y=1)')
class_names = sorted(y[CLASS].unique(),  reverse=True)
print(y[CLASS].value_counts())
y_onehot = pd.get_dummies(y[CLASS])[['Bad']]
print(np.array(np.unique(y_onehot, return_counts=True)).T)

print('X shape:',X1.shape)
X1.head()

Target: Bad (y=1)
Bad     5459
Good    5000
Name: RiskPerformance, dtype: int64
[[   0 5000]
 [   1 5459]]
X shape: (10459, 23)


,ExternalRiskEstimate_bin_WOE,MSinceOldestTradeOpen_bin_WOE,MSinceMostRecentTradeOpen_bin_WOE,AverageMInFile_bin_WOE,NumSatisfactoryTrades_bin_WOE,NumTrades60Ever2DerogPubRec_bin_WOE,NumTrades90Ever2DerogPubRec_bin_WOE,NumTotalTrades_bin_WOE,NumTradesOpeninLast12M_bin_WOE,PercentTradesNeverDelq_bin_WOE,...,PercentInstallTrades_bin_WOE,NetFractionInstallBurden_bin_WOE,NumInstallTradesWBalance_bin_WOE,MSinceMostRecentInqexcl7days_bin_WOE,NumInqLast6M_bin_WOE,NumInqLast6Mexcl7days_bin_WOE,NetFractionRevolvingBurden_bin_WOE,NumRevolvingTradesWBalance_bin_WOE,NumBank2NatlTradesWHighUtilization_bin_WOE,PercentTradesWBalance_bin_WOE
0,1.799,0.086,0.083,0.269,0.166,0.952,-0.021,-0.097,-0.021,1.012,...,-0.503,0.047,0.242,1.223,-0.047,-0.051,-0.088,0.034,-0.601,-0.130
1,1.799,0.549,0.083,1.238,1.999,0.952,-0.053,0.535,-0.021,-0.147,...,0.161,0.047,0.256,1.223,-0.047,-0.051,-0.739,-0.188,0.601,-0.982
2,1.017,0.549,0.083,1.238,0.539,-0.021,-0.021,0.535,0.428,-0.147,...,-0.503,0.147,0.242,1.223,0.170,0.021,0.633,-0.263,-0.601,0.203
3,1.017,0.086,0.083,0.269,-0.086,-0.021,-0.021,-0.377,0.287,0.366,...,-0.145,0.363,0.313,1.223,0.471,0.021,0.633,-0.150,0.541,0.772
4,-1.094,-0.148,0.000,0.000,0.539,-0.021,-0.021,0.116,-0.021,-0.147,...,-0.620,0.363,0.242,1.223,-0.047,-0.051,0.633,-0.188,-0.601,0.203


### AutoKeras

In [36]:
# basic data preparation
from sklearn.preprocessing import LabelEncoder
X = X_onehot.astype('int8')
y = LabelEncoder().fit_transform(y_onehot['Bad'].astype('int8'))

X_onehot_train, X_onehot_test, y_onehot_train, y_onehot_test, = \
train_test_split(X, y, test_size = .25, random_state = 2020, shuffle = True)
print(X_onehot_train.shape, X_onehot_test.shape, y_onehot_train.shape, y_onehot_test.shape)


from autokeras import StructuredDataClassifier
# define the search
search = StructuredDataClassifier(max_trials=15)
# perform the search
search.fit(x=X_onehot_train, y=y_onehot_train, verbose=1)
# evaluate the model
# loss, acc = search.evaluate(X_onehot_test, y_onehot_test, verbose=1)

Trial 15 Complete [00h 02m 33s]
val_accuracy: 0.7097402811050415

Best val_accuracy So Far: 0.7318181991577148
Total elapsed time: 00h 11m 39s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
246/246 [==============================] - 3s 3ms/step - loss: 0.6277 - accuracy: 0.6417
Epoch 2/3
246/246 [==============================] - 1s 3ms/step - loss: 0.5553 - accuracy: 0.7195
Epoch 3/3
246/246 [==============================] - 1s 3ms/step - loss: 0.5374 - accuracy: 0.7308
INFO:tensorflow:Assets written to: .\structured_data_classifier\best_model\assets


In [ ]:
search.evaluate(X_onehot_test, y_onehot_test, verbose=0);

In [ ]:
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()

In [ ]:
# save the best performing model to file
model.save('model_auto.h5',save_format='tf')

### Numeric variables

In [51]:
# basic data preparation
from sklearn.preprocessing import LabelEncoder
X = X.astype('float32')
y = LabelEncoder().fit_transform(y_onehot['Bad'].astype('int8'))

X_onehot_train, X_onehot_test, y_onehot_train, y_onehot_test, = \
train_test_split(X, y, test_size = .25, random_state = 2020, shuffle = True)
print(X_onehot_train.shape, X_onehot_test.shape, y_onehot_train.shape, y_onehot_test.shape)


from autokeras import StructuredDataClassifier
# define the search
search = StructuredDataClassifier(max_trials=15)
# perform the search
search.fit(x=X_onehot_train, y=y_onehot_train, verbose=1)
# evaluate the model
# loss, acc = search.evaluate(X_onehot_test, y_onehot_test, verbose=1)

Trial 15 Complete [00h 00m 15s]
val_accuracy: 0.7032467722892761

Best val_accuracy So Far: 0.7233766317367554
Total elapsed time: 00h 08m 31s
INFO:tensorflow:Oracle triggered exit
Epoch 1/19
246/246 [==============================] - 1s 2ms/step - loss: 0.6983 - accuracy: 0.5768
Epoch 2/19
246/246 [==============================] - 1s 3ms/step - loss: 0.6150 - accuracy: 0.6765
Epoch 3/19
246/246 [==============================] - 1s 3ms/step - loss: 0.6088 - accuracy: 0.6881
Epoch 4/19
246/246 [==============================] - 1s 3ms/step - loss: 0.6048 - accuracy: 0.6998
Epoch 5/19
246/246 [==============================] - 1s 3ms/step - loss: 0.5995 - accuracy: 0.6924
Epoch 6/19
246/246 [==============================] - 1s 3ms/step - loss: 0.5938 - accuracy: 0.6968
Epoch 7/19
246/246 [==============================] - 1s 3ms/step - loss: 0.5851 - accuracy: 0.7067
Epoch 8/19
246/246 [==============================] - 1s 3ms/step - loss: 0.5852 - accuracy: 0.7029
Epoch 9/19
246/246 

In [ ]:
search.evaluate(X_onehot_test, y_onehot_test, verbose=1);

In [ ]:
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()

In [ ]:
# save the best performing model to file
model.save('model_num_auto.h5',save_format='tf')

In [94]:
# scores = model.evaluate(X_onehot_train.values, y_onehot_train)
scores = model.evaluate(X_onehot_train, y_onehot_train, verbose=0)
print("Training Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# scores = model.evaluate(X_onehot_test.values, y_onehot_test)
scores = model.evaluate(X_onehot_test, y_onehot_test, verbose = 0)
print("Test Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# confusion_matrix(y_onehot_test, model.predict_classes(X_onehot_test), sample_weight=None)

Training Set: 
acc: 66.39%
Test Set: 
acc: 66.35%


### WoE variables

In [64]:
# basic data preparation
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import preprocessing
X = df3.astype('float32')
y = LabelEncoder().fit_transform(y_onehot['Bad'].astype('int8'))

X_onehot_train, X_onehot_test, y_onehot_train, y_onehot_test, = \
train_test_split(X, y, test_size = .25, random_state = 2020, shuffle = True)
print(X_onehot_train.shape, X_onehot_test.shape, y_onehot_train.shape, y_onehot_test.shape)


from autokeras import StructuredDataClassifier
# define the search
search = StructuredDataClassifier(max_trials=20)
# perform the search
search.fit(x=X_onehot_train, y=y_onehot_train, verbose=1)
# evaluate the model
# loss, acc = search.evaluate(X_onehot_test, y_onehot_test, verbose=1)

(7844, 23) (2615, 23) (7844,) (2615,)


ImportError: cannot import name 'preprocessing' from 'tensorflow.keras.layers.experimental' (C:\Users\brizio\Anaconda3\lib\site-packages\tensorflow_core\python\keras\api\_v1\keras\layers\experimental\__init__.py)

In [38]:
search.evaluate(X_onehot_test, y_onehot_test, verbose=1);

82/82 [==============================] - 0s 1ms/step - loss: 0.5710 - accuracy: 0.7059


In [39]:
# get the best performing model
model = search.export_model()
# summarize the loaded model
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 23)]              0         
_________________________________________________________________
multi_category_encoding (Mul (None, 23)                0         
_________________________________________________________________
normalization (Normalization (None, 23)                47        
_________________________________________________________________
dense (Dense)                (None, 256)               6144      
_________________________________________________________________
re_lu (ReLU)                 (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224  

In [40]:
# save the best performing model to file
model.save('model_WOE_auto',save_format='tf')

INFO:tensorflow:Assets written to: model_WOE_auto\assets


In [41]:
# scores = model.evaluate(X_onehot_train.values, y_onehot_train)
scores = model.evaluate(X_onehot_train, y_onehot_train, verbose=0)
print("Training Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# scores = model.evaluate(X_onehot_test.values, y_onehot_test)
scores = model.evaluate(X_onehot_test, y_onehot_test, verbose = 0)
print("Test Set:", "\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# confusion_matrix(y_onehot_test, model.predict_classes(X_onehot_test), sample_weight=None)

Training Set: 
accuracy: 73.36%
Test Set: 
accuracy: 70.59%
